In [ ]:
# To-do 
# script allows for duplicates as in if I run process again it would probably redownload a bunch ... 
import pandas as pd
import json
import time
import requests
from datetime import datetime 
import os 
from sutime import SUTime
import dateparser
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from dateutil.relativedelta import relativedelta

In [ ]:
## This is the code for doing limited scrapes, ie 300 for each month of the year, so not an indifferently looping scrape
PUSHSHIFT_REDDIT_URL = "http://api.pushshift.io/reddit"

created = 1323709535

def fetchObjects(**kwargs):
    # Default params values
    
    params = {
        "sort_type": "created_utc",
        "sort": "asc", 
        "size": 300
    }
    
    for key, value in kwargs.items():
        params[key] = value
    #print(params)
    type = "comment"
    if 'type' in kwargs and kwargs['type'].lower() == "submission":
        type = "submission"
        
    #print(PUSHSHIFT_REDDIT_URL + "/" + type + "/search/", params=params, timeout=30)
    r = requests.get(PUSHSHIFT_REDDIT_URL + "/" + type + "/search/", params=params, timeout=30)
    #print(r.url)
    #r = requests.get("https://api.pushshift.io/reddit/search/submission/?subreddit=futurology&sort=asc&sort_type=created_utc&size=12&after=%201323709535&before=%201323820800")
    time.sleep(1)
    if r.status_code == 200:
        response = json.loads(r.text)
        data = response['data']
        #print(data)
        sorted_data_by_id = sorted(data, key=lambda x: int(x['id'], 36))
        return sorted_data_by_id
    else: print("error code: " + str(r.status_code))


def process(max_created_utc, min_created_utc):
    # max_created_utc = 1356998400  # 01/01/2013 @ 12:00am (UTC)
    #min_created_utc = 1323820800
    #max_created_utc = 1323709535  # Update epoch time here if script crashes
    # If script crashes, use tail <filename> to find last created_utc timestamp
    
    max_id = 0
    # file = open("submissions.json","a")  # For submissions
    file = open("full.json", "a")  # For comments
    while 1:
        nothing_processed = True
        # objects = fetchObjects(**kwargs, after=max_created_utc)
        # objects is it hitting the url 
        objects = fetchObjects(subreddit="futurology", type="submission", after=max_created_utc, before=min_created_utc)
        for object in objects:
            id = int(object['id'], 36)
            
            #here we check if the post was created after min_created_utc
            if id > max_id:
                nothing_processed = False
                created_utc = object['created_utc']
                max_id = id
                if created_utc > max_created_utc: 
                    max_created_utc = created_utc
                print(json.dumps(object, sort_keys=True, ensure_ascii=True), file=file)
        
        # add one month to both... 
        # might be worth getting rid of this... 
        return
              
        if nothing_processed: 
            print("nothing")
            return
        max_created_utc -= 1
        time.sleep(.5)

def dateLoop(start, end): 
    start = datetime.fromtimestamp(start)
    end = datetime.fromtimestamp(end)
    while 1:  
        mid = start + relativedelta(months=+1)
        if mid >= end: 
            print(start, end)
            process(int(datetime.timestamp(start)), int(datetime.timestamp(end)))
            return
        print(start, mid)
        process(int(datetime.timestamp(start)), int(datetime.timestamp(mid)))
        start = mid


In [506]:
# Code for infinite scrapes

PUSHSHIFT_REDDIT_URL = "http://api.pushshift.io/reddit"

def fetchObjects(**kwargs):
    # Default params values
    
    params = {
        "sort_type": "created_utc",
        "sort": "asc", 
        "size": 300
    }
    
    for key, value in kwargs.items():
        params[key] = value
    #print(params)
    type = "comment"
    if 'type' in kwargs and kwargs['type'].lower() == "submission":
        type = "submission"
        
    #print(PUSHSHIFT_REDDIT_URL + "/" + type + "/search/", params=params, timeout=30)
    r = requests.get(PUSHSHIFT_REDDIT_URL + "/" + type + "/search/", params=params, timeout=30)
    #print(r.url)
    #r = requests.get("https://api.pushshift.io/reddit/search/submission/?subreddit=futurology&sort=asc&sort_type=created_utc&size=12&after=%201323709535&before=%201323820800")
    time.sleep(1)
    if r.status_code == 200:
        response = json.loads(r.text)
        data = response['data']
        #print(data)
        sorted_data_by_id = sorted(data, key=lambda x: int(x['id'], 36))
        return sorted_data_by_id
    else: print("error code: " + str(r.status_code))


def process(max_created_utc, min_created_utc):
    full = min_created_utc-max_created_utc
    # max_created_utc = 1356998400  # 01/01/2013 @ 12:00am (UTC)
    #min_created_utc = 1323820800
    #max_created_utc = 1323709535  # Update epoch time here if script crashes
    # If script crashes, use tail <filename> to find last created_utc timestamp
    
    max_id = 0
    # file = open("submissions.json","a")  # For submissions
    file = open("infini.json", "a")  # For comments
    while 1:
        nothing_processed = True
        # objects = fetchObjects(**kwargs, after=max_created_utc)
        # objects is it hitting the url 
        objects = fetchObjects(subreddit="futurology", type="submission", after=max_created_utc, before=min_created_utc)
        for object in objects:
            id = int(object['id'], 36)
            
            #here we check if the post was created after min_created_utc
            if id > max_id:
                nothing_processed = False
                created_utc = object['created_utc']
                max_id = id
                if created_utc > max_created_utc: 
                    max_created_utc = created_utc
                    print(100*min_created_utc-max_created_utc/full)
                print(json.dumps(object, sort_keys=True, ensure_ascii=True), file=file)
                
        # add one month to both... 
        # might be worth getting rid of this... 
        # return
              
        if nothing_processed: 
            print("nothing")
            return
        max_created_utc -= 1
        time.sleep(.5)

In [529]:
# Convert date to unix

from dateutil.relativedelta import relativedelta
#x = datetime.now() + relativedelta(months=+2)
x = datetime(2011,2,10)
#y = datetime(2019,2,10)
#print(datetime.now() + relativedelta(months=+2))
print(datetime.timestamp(x))
#print(datetime.timestamp(y))

# Convert unix to date
#x = 1583798400
#datetime.fromtimestamp(x)
#1581292839
#1583794783

1297296000.0


In [522]:
process(1581292800,1583798400)

KeyboardInterrupt: 

In [ ]:
# Load data in as dataframe with only select attributes ... 
fs = pd.read_json('full.json', lines=True)

In [ ]:
fs

In [ ]:
# stop this, hugely inefficient
fd = fs[['title', 'author', 'created_utc', 'num_comments', 'url', 'score', 'selftext']].copy()

In [ ]:
fd["created_utc"] = pd.to_datetime(fd["created_utc"], unit="s")

In [ ]:
fd['yyyy_mm'] = pd.DatetimeIndex(fd['created_utc']).strftime("%Y-%m")

In [ ]:
# sutime set up 
jar_files = os.path.join(os.path.abspath(''), 'jars')
sutime = SUTime(jars=jar_files, mark_time_ranges=True)

In [ ]:
# Pipeline

index = []
dates = []
selftext = []
title  = []
eyes = []

#for i in range(len(fd)):
for i in range(len(fd)):
    print( str(100*i/len(fd)), end="\r")

    for j in sutime.parse(fd['title'][i] + fd['selftext'][i], fd['created_utc'][i].strftime('%Y-%m-%d')):
        if j['type'] == "DATE":
            if dateparser.parse(j['value']) is not None:
                dates.append(dateparser.parse(j['value']))
                index.append(fd["yyyy_mm"][i])
                title.append(fd["title"][i])
                selftext.append(fd["selftext"][i])
                eyes.append(i)

In [494]:
f = {'Years': index, 'Data': dates, 'Title': title}
do = pd.DataFrame(f)
for i in range(len(do)):
    do["Data"][i] = do["Data"][i].strftime("%Y")
do    

,Years,Data,Title
0,2011-12,2011,'A Glorious Dawn' ft Stephen Hawking
1,2011-12,2011,The concept of impossibility is relative
2,2011-12,2011,Message from THE REDDIT(its like 'THEWALMART' ...
3,2011-12,2011,Archaeological Futurism
4,2011-12,2030,The World in 2030 by Michio Kaku
...,...,...,...
3547,2017-10,2049,"In Blade Runner 2049, can a relationship with ..."
3548,2017-10,2018,Flying water taxis coming to Switzerland next ...
3549,2017-10,2025,Italy To Phase Out Coal By 2025
3550,2017-10,2018,So I just read this awesome detailed report by...


In [498]:
# Work out reoccurances of each year
de = do.drop(["Title"], axis=1)
de = de.groupby(de.columns.tolist()).size().reset_index().rename(columns={0:'size'})


In [500]:
# Create after the articles array
news = []
for i in range(len(de)):
    news.append("<br>".join(do.loc[(do["Years"]== de["Years"][i]) & (do["Data"]== de["Data"][i]), "Title"].values))

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(11,4)})

In [164]:
def df_to_plotly(df):
    return {'z': df["size"].tolist(),
            'x': df["Years"].tolist(),
            'y': df["Data"].tolist()}

In [256]:
import plotly.io as pio
pio.templates
pio.templates.default = 'plotly_dark'

In [502]:
import plotly.graph_objects as go

plotly_data = df_to_plotly(de)
data=go.Heatmap(plotly_data, hovertext=news)
fig = go.Figure(data)
#print(fig.layout.template) #go.layout.("dark_background")
#fig.update_xaxes(range=[1.5, 4.5])
fig.update_yaxes(range=[2000, 2080])


fig.show()

In [ ]:
# Notes on what to look for: 
#'Companies' is interesting, can trace is back to them and look at their record of the future ... 
# Technologies: 'flying cars' 'underground road system' 'AI' 'electric cars' 'exoskeletons' 'universal basic income' 'automation' (could this be a keyword)
# 'climate change' 'heat waves' 'flooding' 'tsunami'
# Sentiment: 'Hawking issues familiar warning'
# People
# Speech quotations 
